# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298540054051                   -0.85    5.0
  2   -8.300218016139       -2.78       -1.25    1.9
  3   -8.300448468640       -3.64       -1.86    3.0
  4   -8.300462020152       -4.87       -2.69    1.1
  5   -8.300464356284       -5.63       -3.16    2.8
┌ Warning: Eigensolver not converged
│   iterations =
│    8-element Vector{Int64}:
│     100
│       3
│       3
│       5
│       3
│       4
│       3
│       2
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:34
  6   -8.300464499539       -6.84       -3.28   15.4
  7   -8.300464556437       -7.24       -3.41    4.1
  8   -8.300464598539       -7.38       -3.54    1.0
  9   -8.300464634605       -7.44       -3.78    1.0
 10   -8.300464640402       -8.24       -3.91    1.0
 11   -8.300464643666       -8.49       -4.31    1.1


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67425568857                   -0.70    7.1
  2   -16.67866605237       -2.36       -1.14    1.4
  3   -16.67923204157       -3.25       -1.86    4.2
  4   -16.67928103453       -4.31       -2.77    2.4
  5   -16.67928611053       -5.29       -3.21    5.2
  6   -16.67928621706       -6.97       -3.51    3.5
  7   -16.67928622253       -8.26       -3.98    1.0
  8   -16.67928622427       -8.76       -4.63    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32509260811                   -0.56    6.9
  2   -33.33263257120       -2.12       -1.00    1.0
  3   -33.33412012978       -2.83       -1.74    4.5
  4   -33.33493024776       -3.09       -2.44    3.0
  5   -33.33693299648       -2.70       -2.59   14.0
  6   -33.33694145466       -5.07       -2.90    6.8
  7   -33.33694388357       -5.61       -3.81    2.1
  8   -33.33694390983       -7.58       -4.10    4.5


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298308632872                   -0.85    5.1
  2   -8.300256099607       -2.71       -1.59    1.0
  3   -8.300463050052       -3.68       -2.71    5.2
  4   -8.300463413574       -6.44       -3.24    2.6
  5   -8.300464046273       -6.20       -3.36   11.4
  6   -8.300464596737       -6.26       -3.94    1.2
  7   -8.300464643729       -7.33       -5.08    2.1


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32581116567                   -0.56    7.8
  2   -33.32943682834       -2.44       -1.27    1.1
  3   -25.03506792703   +    0.92       -0.61    8.0
  4   -33.24282725360        0.91       -1.54    5.6
  5   -33.27041772128       -1.56       -1.34    4.2
  6   -33.27547799294       -2.30       -1.65    4.5
  7   -33.14871811814   +   -0.90       -1.42    4.4
  8   -33.33607796340       -0.73       -2.38    3.9
  9   -33.33509771949   +   -3.01       -2.36    2.8
 10   -33.33585578743       -3.12       -2.48    1.8
 11   -33.33637634942       -3.28       -2.62    1.4
 12   -33.33688064299       -3.30       -3.08    3.0
 13   -33.33693040388       -4.30       -3.37    3.8
 14   -33.33693501417       -5.34       -3.44    1.9
 15   -33.33692784514   +   -5.14       -3.45    3.6
 16   -33.33694389220       -4.79       -4.17    2.6


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.